<a href="https://colab.research.google.com/github/alik604/cyber-security/blob/master/Intrusion-Detection/UNSW_NB15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# UNSW-NB15
> 95.5% Accuracy in binary classification

## References 
MLP with pytorch at end
 
 * Data source: https://www.unsw.adfa.edu.au/unsw-canberra-cyber/cybersecurity/ADFA-NB15-Datasets/
 * Sample/starter code: https://github.com/Nir-J/ML-Projects/blob/master/UNSW-Network_Packet_Classification/unsw.py

In [1]:
# !pip install mlxtend catboost xgboost lightgbm

In [2]:
BINARY = False # binary classification or multi-class classification 
%config IPCompleter.greedy=True
import pandas as pd
import seaborn as sns
import numpy as np

import matplotlib as matplot
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import warnings
warnings.filterwarnings("ignore")

from keras import Sequential
from keras.models import Model, load_model
from keras.layers import *
from keras.callbacks import ModelCheckpoint
from keras import regularizers

from sklearn.metrics import *
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,normalize


import xgboost, lightgbm
from mlxtend.classifier import EnsembleVoteClassifier

# Preprocessing (transformation/scaling) 

In [3]:
train = pd.read_csv('https://raw.githubusercontent.com/Nir-J/ML-Projects/master/UNSW-Network_Packet_Classification/UNSW_NB15_training-set.csv')
test = pd.read_csv('https://raw.githubusercontent.com/Nir-J/ML-Projects/master/UNSW-Network_Packet_Classification/UNSW_NB15_testing-set.csv')
combined_data = pd.concat([train, test]).drop(['id'], axis=1) # We will reSplit later

In [4]:
train.head(5)

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,1,0.121478,tcp,-,FIN,6,4,258,172,74.087490,...,1,1,0,0,0,1,1,0,Normal,0
1,2,0.649902,tcp,-,FIN,14,38,734,42014,78.473372,...,1,2,0,0,0,1,6,0,Normal,0
2,3,1.623129,tcp,-,FIN,8,16,364,13186,14.170161,...,1,3,0,0,0,2,6,0,Normal,0
3,4,1.681642,tcp,ftp,FIN,12,12,628,770,13.677108,...,1,3,1,1,0,2,1,0,Normal,0
4,5,0.449454,tcp,-,FIN,10,6,534,268,33.373826,...,1,40,0,0,0,2,39,0,Normal,0


In [5]:
# Contaminsation mean pollution (outliers) in data
tmp = train.where(train['attack_cat'] == "Normal").dropna()
contamination = round(1 - len(tmp)/len(train), 2)
print("train contamination ", contamination)

tmp = test.where(test['attack_cat'] == "Normal").dropna()
print("test  contamination ", round(1 - len(tmp)/len(test),2),'\n')

if contamination > 0.5:
    print(f'Contamination is {contamination}, which is greater than 0.5. Fixing...')
    contamination = round(1-contamination, 2)
    print(f'Contamination is now {contamination}')

train contamination  0.68
test  contamination  0.55 

Contamination is 0.68, which is greater than 0.5. Fixing...
Contamination is now 0.32


In [6]:
le1 = LabelEncoder()
le = LabelEncoder()

vector = combined_data['attack_cat']

print("attack cat:", set(list(vector))) # use print to make it print on single line 

combined_data['attack_cat'] = le1.fit_transform(vector)
combined_data['proto'] = le.fit_transform(combined_data['proto'])
combined_data['service'] = le.fit_transform(combined_data['service'])
combined_data['state'] = le.fit_transform(combined_data['state'])

vector = combined_data['attack_cat']
print('\nDescribing attack_type: ')
print("min", vector.min())
print("max", vector.max())
print("mode", vector.mode(), "Which is,", le1.inverse_transform(vector.mode()))
print("mode", len(np.where(vector.values==6)[0])/len(vector),"%")

attack cat: {'Worms', 'Generic', 'Backdoor', 'Shellcode', 'Reconnaissance', 'Exploits', 'Fuzzers', 'Analysis', 'DoS', 'Normal'}

Describing attack_type: 
min 0
max 9
mode 0    6
dtype: int32 Which is, ['Normal']
mode 0.3609225646458884 %


In [7]:
le1.inverse_transform([0,1,2,3,4,5,6,7,8,9])
combined_data.head(3)

array(['Analysis', 'Backdoor', 'DoS', 'Exploits', 'Fuzzers', 'Generic',
       'Normal', 'Reconnaissance', 'Shellcode', 'Worms'], dtype=object)

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,0.121478,113,0,4,6,4,258,172,74.087490,252,...,1,1,0,0,0,1,1,0,6,0
1,0.649902,113,0,4,14,38,734,42014,78.473372,62,...,1,2,0,0,0,1,6,0,6,0
2,1.623129,113,0,4,8,16,364,13186,14.170161,62,...,1,3,0,0,0,2,6,0,6,0


In [8]:
lowSTD = list(combined_data.std().to_frame().nsmallest(6, columns=0).index)
# this is stupid. suppose a feature has a 1.0 (spearman or pearson) correlation, OR conditional probability, when not 0.... That a very useful feature  

lowCORR = list(combined_data.corr().abs().sort_values('attack_cat')['attack_cat'].nsmallest(3).index) # .where(lambda x: x < 0.005).dropna()
# This might be stupid. A Deep MLP (feed forward neural net) may see patterns

drop = set(lowCORR + lowSTD)
# // drop = {'ackdat', 'ct_ftp_cmd', 'djit', 'is_ftp_login', 'is_sm_ips_ports', 'response_body_len', 'sjit', 'synack', 'tcprtt'}

In [9]:
## OMITTED: statistical feature removal

# print(f'Before {combined_data.shape}')
combined_data_reduced=combined_data # .drop(drop,axis=1)
# print(f'After {combined_data.shape}')

In [10]:
# # transform = list(combined_data_reduced.columns.values[4:])
# transform.append('dur')
# transform.remove('attack_cat')
# # transform min-max norm 
# combined_data_reduced[transform] = combined_data_reduced[transform].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

In [11]:
data_x = combined_data_reduced.drop(['attack_cat','label'], axis=1) # droped label

if BINARY:
    data_y = combined_data_reduced.loc[:, ['label']] 
else: 
    data_y = combined_data_reduced.loc[:, ['attack_cat']]
# del combined_data # free mem
X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, test_size=.20, random_state=42)

In [12]:
data_y.iloc[:, 0].unique()

array([6, 1, 0, 4, 8, 7, 3, 2, 9, 5])

In [13]:
#combined_data_reduced.where(combined_data_reduced['label'] == 1.0).dropna().tail(20)

In [14]:
X_train.shape
y_train.shape
X_test.shape # test is larger... good 
y_test.shape

(206138, 42)

(206138, 1)

(51535, 42)

(51535, 1)

# Benchmark before feature removal

In [15]:
RFC = RandomForestClassifier(n_estimators=150, random_state=42, n_jobs=-1)
ETC = ExtraTreesClassifier(n_estimators=200, random_state=42, n_jobs=-1)
XGB = xgboost.XGBClassifier(n_estimators=75, n_jobs=-1)
if BINARY:
    GBM = lightgbm.LGBMClassifier(objective='binary', n_estimators= 500, n_jobs=-1)
else:
    GBM = lightgbm.LGBMClassifier(objective='multiclass', n_estimators= 500, n_jobs=-1) # multiclass
list_of_CLFs_names, ranking = [], []
list_of_CLFs = [RFC, ETC, XGB, GBM]

for clf in list_of_CLFs:
    _ = clf.fit(X_train,y_train)
    pred = clf.score(X_test,y_test)
    name = str(type(clf)).split(".")[-1][:-2]
    print("Acc: %0.5f for the %s" % (pred, name))

    ranking.append(pred)
    list_of_CLFs_names.append(name)

Acc: 0.82746 for the RandomForestClassifier
Acc: 0.82651 for the ExtraTreesClassifier
[10:40:20] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Acc: 0.83539 for the XGBClassifier
Acc: 0.64139 for the LGBMClassifier


In [16]:
eclf = EnsembleVoteClassifier(clfs=list_of_CLFs, voting='soft', fit_base_estimators=False)
_ = eclf.fit(X_train, y_train)
pred = eclf.score(X_test, y_test)
print("Acc: %0.5f for the %s" % (pred, str(type(eclf)).split(".")[-1][:-2]))


pred = eclf.predict(X_test)
probas = eclf.predict_proba(X_test)
plt.show()

Acc: 0.82348 for the EnsembleVoteClassifier


In [17]:
import catboost

# CBC = catboost.CatBoostClassifier(iterations=3000, eval_metric='AUC', task_type="GPU", devices='0:1', random_seed=42, verbose=False) #  use_best_model=True
CBC = catboost.CatBoostClassifier(eval_metric='AUC', random_seed=42, verbose=False)

_ = CBC.fit(X_train,y_train, eval_set=(X_test, y_test))
pred = CBC.score(X_test,y_test)
name = str(type(CBC)).split(".")[-1][:-2]
print("Acc: %0.5f for the %s" % (pred, name))
list_of_CLFs.append(CBC)

Acc: 0.82406 for the CatBoostClassifier


In [18]:
eclf = EnsembleVoteClassifier(clfs=list_of_CLFs, voting='soft', fit_base_estimators=False)
_ = eclf.fit(X_train, y_train)
pred = eclf.score(X_test, y_test)
print("Acc: %0.5f for the %s" % (pred, str(type(eclf)).split(".")[-1][:-2]))

Acc: 0.82748 for the EnsembleVoteClassifier


In [19]:
from sklearn.feature_selection import RFE
from sklearn.decomposition import PCA, TruncatedSVD, PCA
from sklearn.svm import LinearSVC
n = 10 
X_train.shape

(206138, 42)

### Try RFE

In [20]:
rfe = RFE(DecisionTreeClassifier(), n).fit(X_train, y_train)

desiredIndices = np.where(rfe.support_==True)[0]
whitelist = X_train.columns.values[desiredIndices]
X_train_RFE, X_test_RFE = X_train[whitelist], X_test[whitelist]

print('new shape', X_train_RFE.shape) 

for clf in list_of_CLFs:
    _ = clf.fit(X_train_RFE,y_train)
    pred = clf.score(X_test_RFE,y_test)
    name = str(type(clf)).split(".")[-1][:-2]
    print("Acc: %0.5f for the %s" % (pred, name))

    ranking.append(pred)
    list_of_CLFs_names.append(name)


eclf = EnsembleVoteClassifier(clfs=list_of_CLFs, voting='soft')
_ = eclf.fit(X_train_RFE, y_train)
pred = eclf.score(X_test_RFE, y_test)
print("Acc: %0.5f for the %s" % (pred, str(type(eclf)).split(".")[-1][:-2]))


pred = eclf.predict(X_test_RFE)
probas = eclf.predict_proba(X_test_RFE)
plt.show()

new shape (206138, 10)
Acc: 0.82375 for the RandomForestClassifier
Acc: 0.81873 for the ExtraTreesClassifier
[10:52:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Acc: 0.82439 for the XGBClassifier
Acc: 0.70769 for the LGBMClassifier
Acc: 0.82154 for the CatBoostClassifier
[10:57:19] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Acc: 0.81756 for the EnsembleVoteClassifier


### Try SVD

In [21]:
svd = TruncatedSVD(n_components=n).fit(X_train)
X_train_svd, X_test_svd = svd.transform(X_train), svd.transform(X_test)

for clf in list_of_CLFs:
    _ = clf.fit(X_train_svd, y_train)
    pred = clf.score(X_test_svd, y_test)
    name = str(type(clf)).split(".")[-1][:-2]
    print("Acc: %0.5f for the %s" % (pred, name))

    ranking.append(pred)
    list_of_CLFs_names.append(name)

eclf = EnsembleVoteClassifier(clfs=list_of_CLFs, voting='soft')
_ = eclf.fit(X_train_svd, y_train)
pred = eclf.score(X_test_svd, y_test)
print("Acc: %0.5f for the %s" % (pred, str(type(eclf)).split(".")[-1][:-2]))


pred = eclf.predict(X_test_svd)
probas = eclf.predict_proba(X_test_svd)
plt.show()

Acc: 0.75089 for the RandomForestClassifier
Acc: 0.74868 for the ExtraTreesClassifier
[11:02:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Acc: 0.74109 for the XGBClassifier
Acc: 0.61019 for the LGBMClassifier
Acc: 0.73938 for the CatBoostClassifier
[11:08:19] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Acc: 0.74800 for the EnsembleVoteClassifier


# Benchmark after additional feature removal [Binary Classification]

 **predicting *label*:** 
```
RFE only 
Acc: 0.9259010116 [DecisionTreeClassifier]
Acc: 0.9258389175 [RandomForestClassifier]
Acc: 0.9255957155 [ExtraTreesClassifier]
Acc: 0.9259010116 [Ensemble]
number of features 20
---------------
Acc: 0.9386820522
number of features 10
[ pipeing to SVD 9 dimentions yeilds:  Acc: 0.9221753642 [Ensemble] ; piping to PCA will yeild .921] 
```
```
(64418, 20)
* Acc: 0.9283589040 [DecisionTreeClassifier]
* Acc: 0.9278052314 [RandomForestClassifier]
* Acc: 0.9286848982 [ExtraTreesClassifier]
* Acc: 0.9278000569 [Ensemble]
```
```
(64418, 10)
Acc: 0.9141341751 [DecisionTreeClassifier]
Acc: 0.9145119143 [RandomForestClassifier]
Acc: 0.9141290005 [ExtraTreesClassifier]
Acc: 0.9135028848 [Ensemble]
```




# Lets' try another way

In [22]:
# from sklearn.model_selection import cross_val_score
# from sklearn.ensemble import BaggingClassifier, ExtraTreesClassifier, RandomForestClassifier
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.linear_model import RidgeClassifier
# from sklearn.svm import SVC
# seed = 1075
# np.random.seed(seed)
# # Create classifiers
# rf = RandomForestClassifier()
# et = ExtraTreesClassifier()
# knn = KNeighborsClassifier()
# svc = SVC()
# rg = RidgeClassifier()

In [23]:
# clf_array = [rf, et, knn, svc, rg]
# for clf in clf_array:
#     vanilla_scores = cross_val_score(clf, X_train_RFE, y_train, cv=3, n_jobs=-1)
#     bagging_clf = BaggingClassifier(clf, 
#        max_samples=0.4, max_features=10, random_state=seed)
#     bagging_scores = cross_val_score(bagging_clf, X_test, y_test, cv=3, 
#        n_jobs=-1)
    
#     print("Mean of: {1:.3f}, std: (+/-) {2:.3f} [{0}]".format(clf.__class__.__name__,vanilla_scores.mean(), vanilla_scores.std()))
#     print("Mean of: {1:.3f}, std: (+/-) {2:.3f} [Bagging {0}]\n".format(clf.__class__.__name__,bagging_scores.mean(), bagging_scores.std()))

# Onto Deep Learning 
> I ran SVD/PCA to reduce variance since my goal to "get the best deep learning results possible" 

^ I wrote that 18 months. not sure why that really "matters". now I prefer to given `full` data to my ANNs, and let them `learn` to ignore useless features

#### Now that I try this, I get worse results, I must have done somthing breaking. This is not my goal, `UNSW-NB15_Unsupervised` is. So, ill leave this broken.  

```
Notes: 

sigmoid for binary 

Softmax = non-binary 

```


In [24]:
X = X_train#_RFE
X_t = X_test#_RFE


dim = X.shape[1]
dim

42

In [25]:
# from sklearn.preprocessing import MinMaxScaler
# mms = MinMaxScaler()
# X_train_svd = mms.fit_transform(X_train_svd)
# X_test_svd = mms.transform(X_test_svd)

In [26]:
# classifier = Sequential()
# #First Hidden Layer
# classifier.add(Dense(42, activation='relu', input_dim=dim))

# classifier.add(Dense(64, activation='relu'))
# classifier.add(Dropout(0.07))
# classifier.add(Dense(42, activation='relu' ))
# classifier.add(Dropout(0.07))
# classifier.add(Dense(25, activation='relu'))

# classifier.add(Dense(1, activation='sigmoid'))

# classifier.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy'])

In [27]:
# history = classifier.fit(X,y_train, batch_size=64, epochs=5, validation_data=(X_t,y_test)).history

In [28]:
# eval_model=classifier.evaluate(X, y_train)
# print(eval_model)

# eval_model=classifier.evaluate(X_t, y_test)
# print(eval_model)


# predictions=classifier.predict(X_t)
# predictions =(predictions>0.80)

# mse = np.mean(np.power(X_t - predictions, 2), axis=1)
# error_df = pd.DataFrame({'reconstruction_error': mse,'true_class': y_test.values.reshape(1,-1)[0]})
# error_df.describe()

In [29]:
# plt.plot(history['loss'])
# plt.plot(history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper right');

In [30]:
# fpr, tpr, thresholds = roc_curve(error_df.true_class, error_df.reconstruction_error,pos_label=1)
# roc_auc = auc(fpr, tpr)


# plt.title('Receiver Operating Characteristic')
# plt.plot(fpr, tpr, label='AUC = %0.4f'% roc_auc)
# plt.legend(loc='lower right')
# plt.plot([0,1],[0,1],'r--')
# plt.xlim([-0.001, 1])
# plt.ylim([0, 1.001])
# plt.ylabel('True Positive Rate')
# plt.xlabel('False Positive Rate')
# plt.show();

In [31]:
# precision, recall, th = precision_recall_curve(error_df.true_class, error_df.reconstruction_error,pos_label=1)
# plt.plot(recall, precision, 'b', label='Precision-Recall curve')
# plt.title('Recall vs Precision')
# plt.xlabel('Recall')
# plt.ylabel('Precision')
# plt.show()

In [32]:
## MLP with Pytorch 

# This code has only been run on `UNSW_NB15 - Torch MLP and autoEncoder`

In [33]:
import torch
import torch.nn as nn
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'

input_size = 10
hidden_size = 64 
hidden_size_2 = 64
num_classes = 10

num_epochs = 10
batch_size = 32
learning_rate = 0.001

# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.input_size = input_size # ?? 
        self.l1 = nn.Linear(input_size, hidden_size) 
        self.l2 = nn.Linear(hidden_size, hidden_size_2)  
        self.l3 = nn.Linear(hidden_size_2, num_classes)
        self.relu = nn.ReLU()
        self.elu = nn.ELU()
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        # no activation and no softmax at the end
        return out

In [34]:
model = NeuralNet(input_size, hidden_size, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss() # This criterion combines nn.LogSoftmax() and nn.NLLLoss() in one single class.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

# Train the model
n_total_steps = X_train_RFE_vals.shape[0]

X_train_RFE_vals= X_train_RFE.values
y_train_vals = y_train.values

for epoch in range(num_epochs):
    # for i in range(len(X_train_RFE_vals)//100 + 1): #, batch_size
    for i in range(0, X_train_RFE_vals.shape[0], batch_size):

        x = torch.as_tensor(X_train_RFE_vals[i:i+batch_size], dtype=torch.float).to(device)
        y = torch.as_tensor(y_train_vals[i:i+batch_size], dtype=torch.long).to(device)

        # x.type()
        # y.type()

        outputs = model(x)
        loss = criterion(outputs, y.flatten())
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
#     if (epoch+1) % 10 == 0:
    print(f'Epoch [{epoch+1}/{num_epochs}] | Step [{i+1}/{n_total_steps}] | Loss: {loss.item():.4f}')

NameError: name 'X_train_RFE_vals' is not defined

In [ ]:
## Broken code. See other notebook.
# Test the model
# In test phase, we don't need to compute gradients
# X_test_RFE_vals= X_test_RFE.values
# y_test_vals = y_test.values
# with torch.no_grad():
#     n_correct = 0
#     n_samples = 0 
#     # for i in range(len(X_train_RFE_vals)//100 + 1):   
#     for i in range(0, X_test_RFE_vals.shape[0], 1):
#         x = torch.as_tensor(X_test_RFE_vals[i:i+batch_size], dtype=torch.float).to(device)
#         y = torch.as_tensor(y_test_vals[i:i+batch_size], dtype=torch.long).to(device)
        
#         outputs = model(x)
#         if len(outputs.data) > 0:
#           # max returns (value ,index)
#           _, predicted = torch.max(outputs.data, dim=1)
#           n_samples += y.size(0)
#           n_correct += (predicted == y).sum().item()
#         else:
#           print("what???")
#           print(x, outputs.data)
#     acc = 100.0 * n_correct / (n_samples+1)
#     print(f'Accuracy of the network: {acc} %')

> Accuracy of the network: 79.64%